In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset, dtype=int, drop_first=True) 

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [9]:
indep.shape

(400, 3)

In [10]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
# split into training set and test
# in regression we skip this step in gridsearch method, but in classification we do this in order to get the confusion matrix and classification report.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [12]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
param_grid = {'criterion':['gini', 'entropy', 'log_loss'],
              'splitter':['best', 'random'],
              'max_features':['sqrt', 'log2', None]}

grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2', None],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [14]:
# print best parameter after tuning
# print(grid.best_params_)
clf=grid.cv_results_
grid_predictions=grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, grid_predictions)

# print classification report
from sklearn.metrics import classification_report
clf_report=classification_report(y_test, grid_predictions)

In [15]:
# f1 score is one of the evaluation metrics
# sample to check f1 macro value
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'gini', 'max_features': None, 'splitter': 'best'}: 0.8715363275786964


In [16]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[79  6]
 [11 38]]


In [17]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.88      0.93      0.90        85
           1       0.86      0.78      0.82        49

    accuracy                           0.87       134
   macro avg       0.87      0.85      0.86       134
weighted avg       0.87      0.87      0.87       134



In [18]:
# roc_auc score is also an evaluation metric (a scoring parametr in classification)
# if the roc_auc value is nearly 1, then the model performs well

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.8524609843937575

In [19]:
table=pd.DataFrame.from_dict(clf)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002608,4.901174e-04,0.002723,9.469449e-04,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.799620,0.813511,0.832483,0.851527,0.865054,0.832439,0.023931,8
1,0.002126,1.415877e-03,0.003454,8.591966e-04,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.808927,0.831253,0.796687,0.832483,0.865054,0.826881,0.023417,11
2,0.002010,2.166332e-06,0.002808,7.498516e-04,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.782557,0.850809,0.814409,0.851527,0.886792,0.837219,0.035652,7
3,0.001281,1.066707e-03,0.002613,7.349477e-04,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.774691,0.889022,0.796284,0.847020,0.823140,0.826031,0.039843,13
4,0.001414,3.769467e-04,0.002062,6.256971e-04,gini,None,best,"{'criterion': 'gini', 'max_features': None, 's...",0.832255,0.868752,0.833323,0.906935,0.943041,0.876861,0.042977,1
5,0.001805,9.727046e-04,0.002035,2.336064e-05,gini,None,random,"{'criterion': 'gini', 'max_features': None, 's...",0.829615,0.815037,0.814409,0.865054,0.829124,0.830648,0.018410,9
6,0.001418,9.814506e-04,0.002585,9.205344e-04,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.791752,0.846714,0.833323,0.867097,0.783837,0.824544,0.031974,14
7,0.001669,2.336015e-04,0.001257,1.250110e-03,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.867478,0.795789,0.778067,0.905069,0.850543,0.839389,0.046666,6
8,0.000000,0.000000e+00,0.003987,7.734191e-04,entropy,log2,best,"{'criterion': 'entropy', 'max_features': 'log2...",0.804764,0.831253,0.758030,0.832483,0.905069,0.826320,0.047732,12
9,0.000984,1.205267e-03,0.003270,1.082747e-03,entropy,log2,random,"{'criterion': 'entropy', 'max_features': 'log2...",0.787654,0.758660,0.758030,0.735849,0.869709,0.781980,0.046849,18


In [20]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age: 55
Salary: 55000
Sex Male 0 or 1: 0


In [21]:
Future_Prediction = grid.predict([[age_input,salary_input,sex_male_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
